데이터를 불러와보자 'with 영화진흥원api'

In [62]:
import os
import requests
import json
import calendar
from tqdm import tqdm
from datetime import date, timedelta

key='1b6f2854da67fd41ed15ccc098c08ae0'
movie_info_path='C:/Users/USER/Documents/DataScience/data'

In [63]:
def check_dir(start_date,end_date):    
    start_year = int(start_date.year)
    end_year = int(end_date.year)
    year = start_year
    for i in tqdm(range(start_year,end_year+1)):        
        if(os.path.isdir(movie_info_path+str(year))!=1):
            os.mkdir(movie_info_path+str(year)+'/')
            print("create "+str(year)+" folder") 

        if(os.path.isdir(movie_info_path+str(year)+'_K')!=1):
            os.mkdir(movie_info_path+str(year)+'_K/')
            print("create "+str(year)+"_K folder")   

        if(os.path.isdir(movie_info_path+str(year)+'_F')!=1):
            os.mkdir(movie_info_path+str(year)+'_F/')
            print("create "+str(year)+"_F folder")    
        year += 1

In [64]:
def daily_box_office_to_json(start_date,end_date):    
    tmp_date = start_date
    startDt = int(str(start_date)[0:4]+str(start_date)[5:7]+str(start_date)[8:10])
    endDt = int(str(end_date)[0:4]+str(end_date)[5:7]+str(end_date)[8:10])
    days = end_date - start_date
    for i in tqdm(range(0,days.days+1)):
        targetDt = int(str(tmp_date)[0:4]+str(tmp_date)[5:7]+str(tmp_date)[8:10])

        if(os.path.isfile("C:/Users/USER/Documents/DataScience/data"+str(targetDt)[0:4]+'/'+str(targetDt)+'.json')!=1):
            url='http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key='+key+'&targetDt='+str(targetDt)
            res = requests.get(url)
            text = res.text
            d = json.loads(text)
            with open('data'+str(targetDt)[0:4]+'/'+str(targetDt)+'.json', 'w', encoding="utf-8") as make_file:
                json.dump(d, make_file, ensure_ascii=False, indent="\t")

        if(os.path.isfile("C:/Users/USER/Documents/DataScience/data"+str(targetDt)[0:4]+'_K/'+str(targetDt)+'_K.json')!=1):
            url='http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key='+key+'&targetDt='+str(targetDt)+'&repNationCd=K'
            res = requests.get(url)
            text = res.text
            d = json.loads(text)
            with open('data'+str(targetDt)[0:4]+'_K/'+str(targetDt)+'_K.json', 'w', encoding="utf-8") as make_file:
                json.dump(d, make_file, ensure_ascii=False, indent="\t")

        if(os.path.isfile("C:/Users/USER/Documents/DataScience/data"+str(targetDt)[0:4]+'_F/'+str(targetDt)+'_F.json')!=1):
            url='http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key='+key+'&targetDt='+str(targetDt)+'&repNationCd=F'
            res = requests.get(url)
            text = res.text
            d = json.loads(text)
            with open('data'+str(targetDt)[0:4]+'_F/'+str(targetDt)+'_F.json', 'w', encoding="utf-8") as make_file:
                json.dump(d, make_file, ensure_ascii=False, indent="\t")

        tmp_date += timedelta(1)

In [65]:
start_date = date(2018,2,1)
end_date = date(2018,2,28)
check_dir(start_date,end_date)
daily_box_office_to_json(start_date,end_date)

100%|██████████| 28/28 [01:08<00:00,  2.35s/it]


In [66]:
import pandas as pd
import requests
import datetime
import json
from tqdm import tqdm_notebook
from dateutil.relativedelta import relativedelta

my_date = datetime.date(2018,2,1)

In [31]:
start_date = date(2017,1,1)
targetDt = str(start_date)[0:4]+str(start_date)[5:7]+str(start_date)[8:10]
with open('data/'+str(targetDt)[0:4]+'/'+str(targetDt)+'.json', 'r', encoding="utf-8") as read_file:
    d = json.load(read_file)
d

FileNotFoundError: [Errno 2] No such file or directory: 'data/2017/20170101.json'

In [70]:
#일별 데이터  : csv로 저장 함수 정의 

mykey= '1b6f2854da67fd41ed15ccc098c08ae0' #kobis에서 key 발급
#mykey='4aa9ba2ba5fd74d34df70439d3b483db'
# mykey='aec9e1bc0c849c607bac8a03d6bdb25e' #내 키 일일 조회수 초과해서 예시에 있는 키

def daily_boxoffice(start_date, end_date) :

    final_list = []
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json'

    for single_date in tqdm_notebook(pd.date_range(start_date, end_date)) :
        for rep in ["","_K", "_F"] :            
            dic={
            'key' : mykey,
            'targetDt' : single_date.strftime('%Y%m%d'),
            'itemPerPage' : '10', 
            'repNationCd' : rep #한국영화 K, 외국영화 F
            }

            req = requests.get(url, params = dic)           
            text = req.text
            d = json.loads(text) 

            targetDt = str(single_date)[0:4]+str(single_date)[5:7]+str(single_date)[8:10]
            with open('C:/Users/USER/Documents/DataScience/data'+str(targetDt)[0:4]+rep+'/'+str(targetDt)+rep+'.json', 'r', encoding="utf-8") as read_file:
                d = json.load(read_file)

            for item in d["boxOfficeResult"]["dailyBoxOfficeList"] :
                value_list = []
                key_list = []
                for key, value in item.items() :
                    key_list.append(key)
                    value_list.append(value)

                value_list.append(single_date)
                key_list.append('CurrentDate')

                value_list.append(rep)
                key_list.append('Nation')

                final_list.append(value_list)

    return pd.DataFrame(final_list, columns=key_list)     # final_list에 담긴 내용을 df로 반

In [71]:
start_date = date(2018,2,1)
end_date = date(2018,2,28)
a = daily_boxoffice(start_date, end_date)

A Jupyter Widget

In [37]:
HBox(children=(IntProgress(value=0, max=365), HTML(value='')))

NameError: name 'HBox' is not defined

In [72]:
movie_code_list_all=pd.DataFrame()
for i in tqdm_notebook(range(1)):
    start_date = my_date + relativedelta(months=i)
    end_date = start_date+ relativedelta(months=1) - datetime.timedelta(days=1)      

    daily_boxoffice_data = daily_boxoffice(start_date, end_date)
    startdate=start_date.strftime('%Y%m%d')

    enddate=end_date.strftime('%Y%m%d')

    movie_code_list = pd.DataFrame({'movieCd' :daily_boxoffice_data.movieCd.drop_duplicates(), 'movieNm':daily_boxoffice_data.movieNm.drop_duplicates(), 'director':'','Rate' : 0})


    daily_boxoffice_data.to_csv("daliy_boxoffice_data_"+startdate+"~"+enddate+".csv", index=False)
    movie_code_list.to_csv("movie_code_list_"+startdate+"~"+enddate+".csv", index=False)

    movie_code_list_all = movie_code_list.append(movie_code_list_all)
    movie_code_list_all = movie_code_list_all.drop_duplicates()
    movie_code_list_all.to_csv("movie_code_list_all.csv", index=False)

A Jupyter Widget

A Jupyter Widget